# 单品 AAR 鲁棒库存优化（14天决策周期, Final）

- 基于 Ben-Tal 等（2004）的仿射可调鲁棒（AAR）与 Box 不确定集的对偶等价重构。
- 仅保留“最终版”模型：
  - AAR：u 仿射、y 单一且可仿射（避免双计费），鲁棒包络进入约束，目标含 y 与 ∑δ·n。
  - Static 基线：u 静态、y 静态（不仿射），用于对比体现 AAR 的提升。
- 含完整评估指标与可视化：成本分解、服务水平、随机稳健性、VaR/CVaR。

In [ ]:
# 依赖与环境
import sys, subprocess, os

def ensure(pkg):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for p in ["pandas", "numpy", "pulp", "matplotlib", "seaborn"]:
    ensure(p)

import pandas as pd
import numpy as np
import pulp as pl
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 120)
np.random.seed(42)
print("依赖加载完成")

In [ ]:
# 参数区
PRODUCT = "电解镍"
T_days = 14
K_periods = 8

# 成本
h = 1.0
p = 10.0
c = 0.0
f = 100.0
M = 1e6

# 不确定强度
delta_ratio = 0.2
I1 = 0.0

In [ ]:
# 读取/构造预测并14天聚合
import pandas as pd, numpy as np, os

candidates = ["all_future_predictions.csv"]
forecast_df = None
for name in candidates:
    if os.path.exists(name):
        try:
            df = pd.read_csv(name)
            if {'date','prediction'} <= set(df.columns):
                forecast_df = df.copy()
                break
        except Exception:
            pass

if forecast_df is None:
    days = T_days * K_periods
    date_index = pd.date_range(start="2025-01-01", periods=days, freq="D")
    base = 1000
    season = 200*np.sin(np.arange(days)/7*2*np.pi)
    noise = np.random.normal(0, 50, size=days)
    pred = np.maximum(base + season + noise, 100)
    forecast_df = pd.DataFrame({'date': date_index, 'prediction': pred})

forecast_df['date'] = pd.to_datetime(forecast_df['date'])
forecast_df = forecast_df.sort_values('date')
forecast_df['period'] = ((forecast_df['date'] - forecast_df['date'].min()).dt.days // T_days) + 1
agg = forecast_df.groupby('period', as_index=False)['prediction'].sum().head(K_periods)

d_hat = agg['prediction'].to_numpy()
delta = delta_ratio * d_hat
K = len(d_hat)
print(f"检测到 {len(forecast_df)} 天预测，聚合为 {K} 个14天周期。")

In [ ]:
# AAR Final2：单一仿射 y_k + 对偶鲁棒包络 + 仿射 u_k
m_aar = pl.LpProblem("AAR_Final2", pl.LpMinimize)

v = pl.LpVariable.dicts('v', list(range(1, K+1)), lowBound=0, upBound=1, cat=pl.LpBinary)
u0 = pl.LpVariable.dicts('u0', list(range(1, K+1)), lowBound=0)
A = {}
for k in range(1, K+1):
    for t in range(1, k):
        A[(k,t)] = pl.LpVariable(f"a_{k}_{t}", lowBound=None)

# y(d) = y0 + sum_{t<=k} Y_{k,t} d_t
y0 = pl.LpVariable.dicts('y0', list(range(1, K+1)), lowBound=0)
Y = {}
for k in range(1, K+1):
    for t in range(1, k+1):
        Y[(k,t)] = pl.LpVariable(f"Y_{k}_{t}", lowBound=None)

# 包络松弛变量
Rh = {}; Rp = {}
for k in range(1, K+1):
    for t in range(1, k+1):
        Rh[(k,t)] = pl.LpVariable(f"Rh_{k}_{t}", lowBound=0)
        Rp[(k,t)] = pl.LpVariable(f"Rp_{k}_{t}", lowBound=0)

# 目标中的 z 系数包络（仅来自 c*u 仿射项）
W = pl.LpVariable('W', lowBound=0)
n = pl.LpVariable.dicts('n', list(range(1, K+1)), lowBound=0)

# 仿射 u 的鲁棒上下界
x = {}
for k in range(1, K+1):
    for t in range(1, k):
        x[(k,t)] = pl.LpVariable(f"x_{k}_{t}", lowBound=0)
        m_aar += -x[(k,t)] <= A[(k,t)]
        m_aar += A[(k,t)] <= x[(k,t)]
    if k == 1:
        m_aar += u0[1] >= 0
        m_aar += u0[1] <= M * v[1]
    else:
        m_aar += u0[k] - pl.lpSum(delta[t-1]*x[(k,t)] for t in range(1,k)) >= 0
        m_aar += u0[k] + pl.lpSum(delta[t-1]*x[(k,t)] for t in range(1,k)) <= M * v[k]

# I_k(z) 的常数项与 z 系数
cum_dhat = np.cumsum(d_hat)
for k in range(1, K+1):
    const_k = I1 + pl.lpSum(u0[i] for i in range(1, k+1)) - cum_dhat[k-1]
    for t in range(1, k+1):
        coeff = pl.lpSum(A[(i,t)] for i in range(max(t+1,2), k+1) if (i,t) in A)
        if t == k:
            coeff = coeff + (-1)
        # y ≥ h·I 的包络
        m_aar += -Rh[(k,t)] <= (Y[(k,t)] - h*coeff)
        m_aar += (Y[(k,t)] - h*coeff) <= Rh[(k,t)]
        # y ≥ p·(-I) 的包络
        m_aar += -Rp[(k,t)] <= (-p*coeff - Y[(k,t)])
        m_aar += (-p*coeff - Y[(k,t)]) <= Rp[(k,t)]
    y_const = y0[k] + pl.lpSum(Y[(k,t)]*d_hat[t-1] for t in range(1, k+1))
    m_aar += y_const >= h*const_k + pl.lpSum(delta[t-1]*Rh[(k,t)] for t in range(1, k+1))
    m_aar += y_const >= p*(-const_k) + pl.lpSum(delta[t-1]*Rp[(k,t)] for t in range(1, k+1))

# 目标中的 z 系数仅来自 c*u 仿射项
for t in range(1, K+1):
    coeff_t_obj = pl.lpSum(A[(i,t)] for i in range(max(t+1,2), K+1) if (i,t) in A)
    m_aar += -n[t] <= c * coeff_t_obj
    m_aar += c * coeff_t_obj <= n[t]

# 目标
m_aar += W
m_aar += W >= pl.lpSum(f*v[k] + c*u0[k] + (y0[k] + pl.lpSum(Y[(k,t)]*d_hat[t-1] for t in range(1, k+1))) for k in range(1, K+1)) \
               + pl.lpSum(delta[t-1]*n[t] for t in range(1, K+1))

print("AAR Final2 built. Vars:", len(m_aar.variables()))

In [ ]:
# Static 基线：u 静态，y 静态（不仿射）
m_sta = pl.LpProblem("Static_Robust_Final", pl.LpMinimize)

vs = pl.LpVariable.dicts('v', list(range(1, K+1)), lowBound=0, upBound=1, cat=pl.LpBinary)
us = pl.LpVariable.dicts('u', list(range(1, K+1)), lowBound=0)
ys = pl.LpVariable.dicts('y', list(range(1, K+1)), lowBound=0)

Sh = {}; Sp = {}
for k in range(1, K+1):
    for t in range(1, k+1):
        Sh[(k,t)] = pl.LpVariable(f"Sh_{k}_{t}", lowBound=0)
        Sp[(k,t)] = pl.LpVariable(f"Sp_{k}_{t}", lowBound=0)

W_s = pl.LpVariable('W', lowBound=0)

for k in range(1, K+1):
    m_sta += us[k] >= 0
    m_sta += us[k] <= M * vs[k]

cum_dhat = np.cumsum(d_hat)
for k in range(1, K+1):
    const_k = I1 + pl.lpSum(us[i] for i in range(1, k+1)) - cum_dhat[k-1]
    # I 的 z 系数只有 t=k 为 -1
    m_sta += -Sh[(k,k)] <= -h
    m_sta += -h <= Sh[(k,k)]
    m_sta += -Sp[(k,k)] <= +p
    m_sta += +p <= Sp[(k,k)]
    m_sta += ys[k] >= h*const_k + delta[k-1]*Sh[(k,k)]
    m_sta += ys[k] >= p*(-const_k) + delta[k-1]*Sp[(k,k)]

m_sta += W_s
m_sta += W_s >= pl.lpSum(f*vs[k] + c*us[k] + ys[k] for k in range(1, K+1))

print("Static baseline built. Vars:", len(m_sta.variables()))

In [ ]:
# 求解两模型
res_aar = m_aar.solve(pl.PULP_CBC_CMD(msg=False, timeLimit=180))
res_sta = m_sta.solve(pl.PULP_CBC_CMD(msg=False, timeLimit=120))
print("AAR:", pl.LpStatus[res_aar])
print("Static:", pl.LpStatus[res_sta])

v_sol = np.array([pl.value(v[k]) for k in range(1, K+1)])
u0_sol = np.array([pl.value(u0[k]) for k in range(1, K+1)])
A_mat = np.zeros((K,K))
for k in range(1, K+1):
    for t in range(1, k):
        A_mat[k-1, t-1] = pl.value(A[(k,t)]) if (k,t) in A else 0.0

y0_sol = np.array([pl.value(y0[k]) for k in range(1, K+1)])
Y_mat = np.zeros((K,K))
for k in range(1, K+1):
    for t in range(1, k+1):
        Y_mat[k-1, t-1] = pl.value(Y[(k,t)]) if (k,t) in Y else 0.0

vs_sol = np.array([pl.value(vs[k]) for k in range(1, K+1)])
us_sol = np.array([pl.value(us[k]) for k in range(1, K+1)])
ys_sol = np.array([pl.value(ys[k]) for k in range(1, K+1)])

In [ ]:
# 评估与对比（保留完整指标）

def affine_policy_orders(u0_vec, A_mat, z_vec):
    K = len(u0_vec)
    u = u0_vec.copy().astype(float)
    for k in range(1, K):
        u[k] = u0_vec[k] + np.dot(A_mat[k, :k], z_vec[:k])
    return np.maximum(u, 0.0)


def simulate_trajectory(u_vec, v_vec, d_hat_vec, z_vec, I1, costs):
    f, c, h, p = costs
    K = len(u_vec)
    I = np.zeros(K)
    d_real = d_hat_vec + z_vec

    I[0] = I1 + u_vec[0] - d_real[0]
    for k in range(1, K):
        I[k] = I[k-1] + u_vec[k] - d_real[k]

    y_pos = np.maximum(I, 0)
    y_neg = np.maximum(-I, 0)

    fixed = f * np.sum((v_vec > 0.5).astype(int))
    variable = c * np.sum(u_vec)
    holding = h * np.sum(y_pos)
    penalty = p * np.sum(y_neg)
    total = fixed + variable + holding + penalty

    service_rate = np.mean(I >= 0)
    stockouts = int(np.sum(I < 0))

    return dict(total=total, fixed=fixed, variable=variable, holding=holding, penalty=penalty,
                service_rate=service_rate, stockouts=stockouts, I_series=I)


def evaluate(u0_vec, A_mat, v_vec, d_hat_vec, delta_vec, I1, costs, num_mc=300, seed=2025):
    rng = np.random.default_rng(seed)
    z0 = np.zeros_like(delta_vec)
    nom = simulate_trajectory(affine_policy_orders(u0_vec, A_mat, z0), v_vec, d_hat_vec, z0, I1, costs)

    z_plus = +delta_vec
    z_minus = -delta_vec
    plus = simulate_trajectory(affine_policy_orders(u0_vec, A_mat, z_plus), v_vec, d_hat_vec, z_plus, I1, costs)
    minus = simulate_trajectory(affine_policy_orders(u0_vec, A_mat, z_minus), v_vec, d_hat_vec, z_minus, I1, costs)

    mc = []
    for _ in range(num_mc):
        z = rng.uniform(low=-delta_vec, high=+delta_vec)
        mc.append(simulate_trajectory(affine_policy_orders(u0_vec, A_mat, z), v_vec, d_hat_vec, z, I1, costs))

    def agg(key):
        arr = np.array([it[key] for it in mc])
        return dict(mean=float(arr.mean()), std=float(arr.std()), p90=float(np.quantile(arr,0.9)),
                    p95=float(np.quantile(arr,0.95)), p99=float(np.quantile(arr,0.99)))

    totals = np.array([it['total'] for it in mc])
    var95 = float(np.quantile(totals, 0.95))
    cvar95 = float(totals[totals>=var95].mean())

    return dict(nominal=nom, plus=plus, minus=minus,
                mc=dict(total=agg('total'), service=agg('service_rate'), stockouts=agg('stockouts'),
                        VaR95=var95, CVaR95=cvar95))

costs_tuple = (f, c, h, p)

# AAR 评估（u 仿射）；Static 评估（u 静态）
aar_eval = evaluate(u0_sol, A_mat, v_sol, d_hat, delta, I1, costs_tuple)
sta_eval = evaluate(us_sol, None, vs_sol, d_hat, delta, I1, costs_tuple)

summary = pd.DataFrame([
    dict(method='AAR', nominal_total=aar_eval['nominal']['total'], nominal_service=aar_eval['nominal']['service_rate'],
         worst_plus_total=aar_eval['plus']['total'], worst_minus_total=aar_eval['minus']['total'],
         MC_mean_total=aar_eval['mc']['total']['mean'], MC_std_total=aar_eval['mc']['total']['std'],
         MC_VaR95=aar_eval['mc']['VaR95'], MC_CVaR95=aar_eval['mc']['CVaR95'],
         MC_mean_service=aar_eval['mc']['service']['mean']),
    dict(method='Static', nominal_total=sta_eval['nominal']['total'], nominal_service=sta_eval['nominal']['service_rate'],
         worst_plus_total=sta_eval['plus']['total'], worst_minus_total=sta_eval['minus']['total'],
         MC_mean_total=sta_eval['mc']['total']['mean'], MC_std_total=sta_eval['mc']['total']['std'],
         MC_VaR95=sta_eval['mc']['VaR95'], MC_CVaR95=sta_eval['mc']['CVaR95'],
         MC_mean_service=sta_eval['mc']['service']['mean'])
])
summary

In [ ]:
# 名义场景成本分解对比
nom_breakdown = pd.DataFrame([
    dict(method='AAR', fixed=aar_eval['nominal']['fixed'], variable=aar_eval['nominal']['variable'],
         holding=aar_eval['nominal']['holding'], penalty=aar_eval['nominal']['penalty'], total=aar_eval['nominal']['total']),
    dict(method='Static', fixed=sta_eval['nominal']['fixed'], variable=sta_eval['nominal']['variable'],
         holding=sta_eval['nominal']['holding'], penalty=sta_eval['nominal']['penalty'], total=sta_eval['nominal']['total'])
])
nom_breakdown

In [ ]:
# 随机场景总成本分布对比
rng = np.random.default_rng(2026)
N = 400
AAR_totals, STA_totals = [], []
for _ in range(N):
    z = rng.uniform(low=-delta, high=+delta)
    AAR_totals.append(simulate_trajectory(affine_policy_orders(u0_sol, A_mat, z), v_sol, d_hat, z, I1, costs_tuple)['total'])
    STA_totals.append(simulate_trajectory(affine_policy_orders(us_sol, None, z), vs_sol, d_hat, z, I1, costs_tuple)['total'])

plt.figure(figsize=(10,5))
plt.hist(AAR_totals, bins=24, alpha=0.6, label='AAR total')
plt.hist(STA_totals, bins=24, alpha=0.6, label='Static total')
plt.axvline(np.mean(AAR_totals), color='C0', linestyle='--', label='AAR mean')
plt.axvline(np.mean(STA_totals), color='C1', linestyle='--', label='Static mean')
plt.title('随机Box场景总成本分布（AAR vs Static）')
plt.legend()
plt.show()